In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.stats import sigma_clip
from transitleastsquares import (
    transitleastsquares,
    cleaned_array,
    catalog_info,
    transit_mask
    )
import pandas as pd
from OrionPY import Tools
import batman







In [2]:
# Loading in the original data for obj_ids and rebinned data
hdu = Tools()
data = np.load('/data/jpr64/rebinned_data_test.npy', allow_pickle = True)





/home/jpr64/orion/Orion_main/OrionPY.py:71: RuntimeWarning: invalid value encountered in log10
  self.mag = -2.5*np.log10(self.flux)


In [5]:
# Putting data into lists
lc = [x for x in data]
obj_ids = [x for x in hdu.hdu[1].data['obj_id']]

idx = [i for i, j  in enumerate(lc) if not j.shape == (0,3)]

lc = [lc[i] for i in idx]
obj_ids = [obj_ids[i] for i in idx]


8925 8925


/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [ ]:
def magnitude(lc):
    return -2.5* np.log10(lc[:,1]) + 20.2
def rms(magnitudes):
    return np.sqrt(np.mean(np.square(magnitudes - np.nanmean(magnitudes))))




In [ ]:
# Calculate magnitudes and rms values for selction

magnitudes = [magnitude(x) for x in lc]
mean_mag = [np.nanmean(x) for x in magnitudes]
rms_mag = [rms(x) for x in magnitudes]

mean_mag = np.array(mean_mag)
rms_mag = np.array(rms_mag)


In [ ]:
rms_mag

In [ ]:
# Selecting a suitably quiet lightcurve
idx_2 = np.where(rms_mag < 10**(-2))[0]

lc_low  = [lc[x] for x in idx_2]
obj_ids_low = [obj_ids[x] for x in idx_2]
magnitudes_low = [magnitudes[x] for x in idx_2]
rms_mag = [rms_mag[x] for x in idx_2]


In [ ]:
object_num = 1045


test = lc_low[object_num]
flux = test[:,1]
flux /= np.nanmean(flux)
time = test[:,0]
print(rms_mag[object_num])


In [ ]:
print(rms)

In [ ]:
plt.scatter(time, flux)

In [ ]:
# Celaning of nan and negative values
time, flux = cleaned_array(time, flux)

# Use batman to create transits
ma = batman.TransitParams()
ma.t0 = time[0]  # time of inferior conjunction; first transit is X days after start
ma.per = 7.69 # orbital period
ma.rp = 0.1  # 1% dip planet radius (in units of stellar radii)
ma.a = 8.47 # semi-major axis (in units of stellar radii)
ma.inc = 90  # orbital inclination (in degrees)
ma.ecc = 0  # eccentricity
ma.w = 90  # longitude of periastron (in degrees)
ma.u = []  # limb darkening coefficients
ma.limb_dark = "uniform"  # limb darkening model
m = batman.TransitModel(ma, time)  # initializes model
synthetic_signal = m.light_curve(ma)  # calculates light curve

# Sigma clip the data and then run the TLS search
flux_transit = flux * synthetic_signal 
flux_sigma_clipped = sigma_clip(flux_transit, sigma_lower = float('inf'), sigma_upper= 4)

model = transitleastsquares(time, flux_sigma_clipped)
results = model.power(period_min = 1, period_max = 11, limb_dark = 'linear', u =[0], use_threads = 4)
    
results["obj_id"] = obj_ids[object_num]



In [ ]:
plt.plot(time, flux_sigma_clipped)



In [ ]:
print('Period', format(results.period, '.5f'), 'd')
print(len(results.transit_times), 'transit times in time series:', \
        ['{0:0.5f}'.format(i) for i in results.transit_times])
print('Transit depth', format(results.depth, '.5f'))
print('Best duration (days)', format(results.duration, '.5f'))
print('Signal detection efficiency (SDE):', results.SDE)






In [ ]:
plt.figure()
ax = plt.gca()
ax.axvline(results.period, alpha=0.4, lw=3)
for n in range(2, 10):
    ax.axvline(n*results.period, alpha=0.4, lw=1, linestyle="dashed")
    ax.axvline(results.period / n, alpha=0.4, lw=1, linestyle="dashed")
plt.xlim(np.min(results.periods), np.max(results.periods))
plt.ylabel(r'SDE')
plt.xlabel('Period (days)')
plt.plot(results.periods, results.power, color='black', lw=0.5);

In [ ]:
plt.figure()
plt.plot(
    results.model_folded_phase,
    results.model_folded_model,
    color='red')
plt.scatter(
    results.folded_phase,
    results.folded_y,
    color='blue',
    s=10,
    alpha=0.5,
    zorder=2)
plt.xlim(0, 1)
plt.xlabel('Phase')
plt.ylabel('Relative flux');
